Hypothesis: The number of calls is correlated to the problems the district is facing. 
More calls - more problems or is there misrepresentation, i.e lack of awarness in areas with less calls. The problems can be quantified from the suicide data 

In [94]:
import urllib
import json
import psycopg2
import pandas as pd

import numpy as np
import requests
import json
import urllib
import json
import pandas as pd
import seaborn as sns

from matplotlib.ticker import FuncFormatter

import matplotlib.pyplot as plt
%matplotlib inline

In [121]:
f = open('../kcc_key.txt', 'r')
key = f.readlines()[0]

In [5]:
# def connect():
#     """ Connect to the PostgreSQL database server """
    
#     conn = None
#     try:

#         # connect to the PostgreSQL server
#         print('Connecting to the PostgreSQL database...')
#         conn = psycopg2.connect(host="localhost",database="KisanCallCenter", 
#                                 user="postgres", password=key)
 
#         # create a cursor
#         cur = conn.cursor()
        
#  # execute a statement
#         print('PostgreSQL database version:')
#         cur.execute('SELECT version()')
 
#         # display the PostgreSQL database server version
#         db_version = cur.fetchone()
#         print(db_version)
       
#      # close the communication with the PostgreSQL
#         cur.close()
#     except (Exception, psycopg2.DatabaseError) as error:
#         print(error)
#     finally:
#         if conn is not None:
#             conn.close()
#             print('Database connection closed.')

In [9]:
def connect_kcc():
    """ Connect to the Kisan Call Center database server """
        
    conn = None
    try:

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(host="localhost",database="KisanCallCenter", 
                                user="postgres", password=key)
 
        # create a cursor
        cur = conn.cursor()
        print "Connected"
    
        return conn, cur
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

def execute_query(conn, cur, sql, value):
    
    cur.execute(sql, value)
        
    # get the generated id back
    call_id = cur.fetchone()[0]
    return call_id

def commit_close_connection(conn, cur):
    
    cur.close()
    # commit the changes
    conn.commit()
    
    if conn is not None:
        conn.close()
        print('Database connection closed.')

In [25]:
def create_tables():
    """ create tables in the PostgreSQL database"""
    command = (
        """
        CREATE TABLE kcc (
            call_id serial PRIMARY KEY,
            BlockName text,
            Category text,
            CreatedOn timestamp,
            Crop text,
            DistrictName text,
            KCCAns text,
            Query text,
            QueryType text,
            Season text,
            Sector text,
            StateName text
        )
        """
    )
    
    conn = None
    try:
        
        # connect to the PostgreSQL server
        conn = psycopg2.connect(host="localhost",database="KisanCallCenter", 
                                user="postgres", password=key)
        cur = conn.cursor()
        
        # create table
        cur.execute(command)
        # close communication with the PostgreSQL database server
        
        cur.close()
        # commit the changes
        conn.commit()
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    
    finally:
        if conn is not None:
            conn.close()

In [26]:
create_tables()

In [27]:
# def insert_kcc(row):
    
#     sql = """INSERT INTO kcc(BlockName, Category, CreatedOn, Crop, DistrictName, \
# Query, KCCAns, QueryType, Season, Sector, StateName) \
# VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) RETURNING call_id;"""
    
#     conn = None
#     call_id = None
#     try:
        
#         # connect to the PostgreSQL server
#         conn = psycopg2.connect(host="localhost",database="KisanCallCenter", 
#                                 user="postgres", password=key)
#         # create a new cursor
#         cur = conn.cursor()
        
#         # execute the INSERT statement
#         cur.execute(sql, row)
        
#         # get the generated id back
#         call_id = cur.fetchone()[0]
#         # commit the changes to the database
#         conn.commit()
#         # close communication with the database
#         cur.close()
#         return call_id
#     except (Exception, psycopg2.DatabaseError) as error:
#         print(error)
#     finally:
#         if conn is not None:
#             conn.close()

In [27]:
'''
/**
 * Author: Sameer Waskar (dev.wsameer@gmail.com), Vishwajeet Shelar (vys217@nyu.edu)
 * Desc: Download district wise and month wise queries of farmers in Kisan Call Centre (KCC) during 2015, 2016
 *       The files are downloaded in json format.
 *       File name format: [Month]_[District Code]_[Year]
 *       # http://dackkms.gov.in/Account/API/kKMS_QueryData.aspx?StateCD=03&DistrictCd=0309&Month=10&Year=2016
 */
'''

def download_kcc():

    # @type {String} To store the base URL to hit
    baseUrl = "http://dackkms.gov.in/Account/API/kKMS_QueryData.aspx?"

    # @type {Dictionary} To store the query params
#     payload = {'StateCD': '11', 'DistrictCd': '1111', 'Month': '01', 'Year': '2015'}

    # @type {Dictionary} To store month names
    monthSet = {
                    '01': 'Jan',
                    '02': 'Feb',
                    '03': 'Mar',
                    '04': 'Apr',
                    '05': 'May',
                    '06': 'Jun',
                    '07': 'Jul',
                    '08': 'Aug',
                    '09': 'Sep',
                    '10': 'Oct',
                    '11': 'Nov',
                    '12': 'Dec'
                  }

    # @type {List} To store years to download data
    years = ['2015', '2016']
#     years = ['2015']

    # @type {List} To store months to download data
    months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
#     months = ['01']
  
    conn, cur = connect_kcc()
    query = """INSERT INTO kcc(BlockName, Category, CreatedOn, Crop, DistrictName, \
Query, KCCAns, QueryType, Season, Sector, StateName) \
VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) RETURNING call_id;"""
    
    for index, year in enumerate(years):
        for month in months:
            print "\nDownloading data for the month of {}, {}".format(monthSet[month], year),
            distRange = [str(x).zfill(2) for x in range(1, 37)]
            for districtCode in distRange:
                print "\nDownloading data for District #{}\n".format(districtCode),
                payload = {
                            'StateCD': '11',
                            'DistrictCd': '11'+districtCode,
                            'Month': month,
                            'Year': year
                          }
 
                params = "StateCD=" + payload['StateCD'] + "&DistrictCd=" + \
payload['DistrictCd'] + "&Month=" + payload['Month'] + "&Year=" + payload['Year']
                
                url = baseUrl + params
                uh = urllib.urlopen(url)
                data = uh.read()

                try: 
                    js = json.loads(str(data))
                except:
                    print "No data"
                else:    
                    if js['ResponseCode'] == '0':
                        print "No data"
                    else:
                        
                        for call in range(len(js['data'])):
                                                       
                            row =  (str(js['data'][call]['BlockName'].strip()), 
                                    str(js['data'][call]['Category'].strip()) ,
                                    str(js['data'][call]['CreatedOn']), 
                                    str(js['data'][call]['Crop'].strip()),  
                                    str(js['data'][call]['DistrictName'].strip()), 
                                    js['data'][call]['QueryText'],
                                    js['data'][call]['KCCAns'], 
                                    str(js['data'][call]['QueryType'].strip()) ,
                                    str(js['data'][call]['Season'].strip()), 
                                    str(js['data'][call]['Sector'].strip()), 
                                    str(js['data'][call]['StateName'].strip()))
                            
                            call_id = execute_query(conn, cur, sql=query, value=row)
                            print '\r',str(call_id),
                            conn.commit()

    print("\nProgram terminated normally")
    
    return conn, cur
    commit_close_connection(conn, cur)

In [28]:
conn, cur = download_kcc()

Connecting to the PostgreSQL database...
Connected

3 
142 
214 
241 
283 
2750       
3318  
4684      
7312         
8319     
8968     
9813   
12320        
12825  
14379       
15641    
16574   
17928     
19232     
20180         
20882   
22682       
23616   
24527   
25387    
25616 
25985  
26122 
26313  
26344 
27238   
27321 
28057   
28117 
28139 
No data

28140 
28275 
28324 
28368 
28409 
30509       
31129   
32416      
34818          
35966    
36598  
37368   
39774           
40324  
41609     
42659       
43473    
44892     
46060    
46815    
47370    
48985     
49867    
50701   
51491   
51714 
52076 
52222  
52427 
52472 
53264   
53372 
54053   
54136 
54158 
No data

54159 
54280 
54326 
54371 
54401 
57579           
58324    
60219         
63440                 
64658        
65537    
66955     
70461           
71326    
73077       
74550       
75799    
77456       
78993      
80154     
81029  
83620        
84982      
86086    
87152    
8741

165922 
165977 
165994 
166020 
166039 
168064       
168541  
170163      
172379           
173225   
173779   
174444   
176482     
177001  
178794      
180434      
182249       
183928     
185856       
186854   
187680   
190204       
191348     
192311   
193777     
194239  
194694  
194868 
195089 
195141 
196264     
196338 
197548    
197632 
197655 
No data

No data

197696 
197712 
197733 
197745 
199948       
200482  
202426       
204305     
204895  
205308  
205841  
207682       
208015  
210110        
212381        
214673        
216664       
219406           
220538     
221538      
225162            
226720     
228055   
229863       
230383      
230835  
230966 
231236    
231285 
232997        
233056  
235090       
235168 
235175 
No data

235177 
235226 
235242 
235257 
235270 
237570       
238279   
240192      
242076       
242675   
243172   
243650   
245538      
245876 
248369        
250835        
252960      
254852      
257745         


345939 
No data

345942 
346071  
346112 
346141 
346169 
349398        
350289   
351854     
355322           
356707      
357419   
358448   
361989           
362494  
364771        
366511     
368129      
370179       
372066      
373666      
374849    
377360         
378973     
380379    
381792    
382340  
382877  
383174 
383598  
383683 
385129      
385294 
386798     
386954 
386978 
No data

386980 
387153 
387224 
387263 
387301 
390259         
391213    
392712     
396603            
398232       
399098    
400091    
403997            
404560   
406764          
408775      
410340    
412683        
414457      
415908     
417029    
419356       
421017      
422348    
423785     
424294  
424927  
425258 
425666  
425738 
426996     
427107 
428540     
428698 
428719 
No data

428720 
428910 
428965 
429033 
429087 
431268       
432157   
433551    
436870           
438298    
439125   
439897   
442862           
443418  
445590        
447233       


537668  
538697                     
538958 
539022 
No data

No data

539169 
539227 
539274 
539327 
541165     
541768  
542701    
545014       
546378     
547058  
547846   
550011      
550733   
552203      
553313     
554542      
555852     
557203    
558202   
559027    
560617       
561831      
562915      
563910    
564229  
564736  
564939 
565240 
565283 
566168     
566255 
567120  
567280 
567385 
No data

567386 
567416 
567435 
567460 
567465 
568120   
568376  
568733  
569579   
569918 
570097 
570372 
571056  
571278 
571822  
572217  
572577  
573049  
573528  
573806  
574055 
574548  
574852  
575146 
575485 
575572 
575728 
575800 
575859 
575874 
576147 
576172 
576399 
576431 
576441 
No data

576442 
576460 
576479 
576487 
576498 
577428    
577678  
578492   
579555    
579922  
580125 
580378 
581296    
581503   
582599     
583306    
583952   
584722   
585686      
586148   
586499  
587575    
588160  
588701  
589509   
589781  
590067  
59013

652955 
652965 
No data

No data

652993 
653003 
653014 
653026 
655082        
655616  
656985       
659298         
659937   
660318  
660821  
663345         
663642 
666233         
668773        
670986       
673395       
675970      
677084    
677955   
680829            
682332     
683584     
685338     
685830  
686423  
686592 
686935  
686996 
688621    
688709 
690281    
690369 
690392 
No data

No data

690428 
690444 
690459 
690475 
692586      
693036  
694152    
696401       
697209  
697682   
698309  
701129        
701452  
703251       
704807      
706318     
707873     
710206        
711357    
712438     
714471      
715706    
716818    
718436      
718918   
719451  
719579  
720013   
720112 
721503     
721584 
722925    
723012 
723039 
No data

No data

723086 
723096 
723105 
723128 
724781     
725149  
726108   
728312       
729120    
729574  
730183   
732675         
732978 
734413    
735811    
736999    
738429     
740137     
741160